<a href="https://colab.research.google.com/github/kerim371/Julia-Devito-Inversion-JUDI-examples-in-Colab/blob/main/JUDI_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this in Colab you either have to use this script or download it and then upload to Colab. You cannot simply copy this code and past it to your Colab session - this won't let you use Julia. You can open **.pynb** document in textual editor and see at top of the document how it Julia is et up and change its version as well if you need.

# Julia installation

In [ ]:
# Julia installation
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

# JUDI and DEVITO installation

In [ ]:
!pip install devito matplotlib # or `pip3 install --user devito matplotlib`
!wget ftp://slim.gatech.edu/data/SoftwareRelease/WaveformInversion.jl/2DFWI/overthrust_2D.segy
!wget ftp://slim.gatech.edu/data/SoftwareRelease/WaveformInversion.jl/2DFWI/overthrust_2D_initial_model.h5

In [ ]:
import os
os.environ['PYTHON'] = '/usr/local/bin/python'

!julia -e 'using Pkg; Pkg.add("Pkg")'
!julia -e 'using Pkg; Pkg.Registry.add(RegistrySpec(url="https://github.com/slimgroup/SLIMregistryJL.git"))'
!julia -e 'using Pkg; Pkg.develop("JUDI")'

'cd ' command should be in separated cell

In [ ]:
cd ~/.julia/dev/JUDI

In [ ]:
!julia deps/install_global.jl
#!julia --project -e 'using Pkg;Pkg.test(coverage=false)' # uncomment this if you want to run test

# JUDI example

In [ ]:
cd /content

Until now we have been using ipython. Now we are going to use Julia. To achieve it go to Colab's menu bar and select **Edit** and select **Notebook settings** from the drop down. Select *Julia 1.5.0* as the runtime and *GPU* as the hadware accelerator. If necessary reload web-page (F5) and change **Notebook settings**

In [ ]:
using Pkg

Pkg.add("PyPlot")
Pkg.add("HDF5")
Pkg.add("SegyIO")

using PyPlot, HDF5, SegyIO, JUDI.TimeModeling, JUDI.SLIM_optim, Statistics, Random

# Load starting model
n, d, o, m0 = read(h5open("overthrust_2D_initial_model.h5", "r"), "n", "d", "o", "m0")
model0 = Model((n[1], n[2]), (d[1], d[2]), (o[1], o[2]), m0)	# need n, d, o as tuples and m0 as array

# Bound constraints
vmin = ones(Float32, model0.n) .+ 0.3f0
vmax = ones(Float32, model0.n) .+ 5.5f0
mmin = vec((1f0 ./ vmax).^2)	# convert to slowness squared [s^2/km^2]
mmax = vec((1f0 ./ vmin).^2)

# Load segy data
block = segy_read("overthrust_2D.segy")
dobs = judiVector(block)

# Set up wavelet
src_geometry = Geometry(block; key="source", segy_depth_key="SourceDepth")	# read source position geometry
wavelet = ricker_wavelet(src_geometry.t[1], src_geometry.dt[1], 0.008f0)	# 8 Hz wavelet
q = judiVector(src_geometry, wavelet)

In [6]:
# Optimization parameters
fevals = 20	# number of function evaluations
batchsize = 20	# number of sources per iteration
fvals = zeros(21)
opt = Options(optimal_checkpointing = false)    # set to true to enable checkpointing

# Objective function for minConf library
count = 0
function objective_function(x)
	model0.m = reshape(x, model0.n);

	# fwi function value and gradient
	i = randperm(dobs.nsrc)[1:batchsize]
	fval, grad = fwi_objective(model0, q[i], dobs[i]; options=opt)
	grad = reshape(grad, model0.n); grad[:, 1:21] .= 0f0	# reset gradient in water column to 0.
	grad = .1f0*grad/maximum(abs.(grad))	# scale gradient for line search

	global count; count += 1; fvals[count] = fval
    return fval, vec(grad)
end

# FWI with SPG
ProjBound(x) = median([mmin x mmax], dims=2)	# Bound projection
options = spg_options(verbose=3, maxIter=fevals, memory=3)
x, fsave, funEvals= minConf_SPG(objective_function, vec(model0.m), ProjBound, options)

LoadError: ignored

In [ ]:
figure(); imshow(sqrt.(1./adjoint(m0))); title("Initial model")
figure(); imshow(sqrt.(1./adjoint(reshape(x, model0.n)))); title("FWI")
figure(); plot(fvals); title("Function value")



---


# You can run this to check if Julia is running and its GPU perfomance




In [ ]:
VERSION

The main reason we are interested in running Julia on Colab is the GPU functionality. So we can benchmark the performance on the GPU.<br>**The next two cells are optional and are for people using it for the first time**

In [ ]:
using Pkg
Pkg.add(["BenchmarkTools", "CuArrays"])
using BenchmarkTools,CuArrays

In [ ]:
mcpu = rand(2^10, 2^10)
@benchmark mcpu*mcpu

println("The CuArrray operation should take around 1 ms(excluding CUDA downloading time which is a one time process), and should be much faster. If so, the GPU is working.")
mgpu = cu(mcpu)
@benchmark CuArrays.@sync mgpu*mgpu